In [1]:
import warnings
import numpy as np
import pandas as pd
#from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config

warnings.filterwarnings("ignore")

#import sklearn
#sklearn.show_versions()
#sklearn.__version__
set_config(display='diagram')

In [2]:
data = pd.read_csv('data/train.csv', index_col=0)

In [3]:
import flaml
flaml.__version__

'0.10.0'

In [4]:
from sklearn.preprocessing import MinMaxScaler
from numpy import log
from datetime import date, datetime

def feature_generation(data):

    features = data.columns

    data.time = pd.to_datetime(data.time)

    data.loc[:, 'hour'] = data.time.dt.hour
    data.loc[:, 'day_of_week'] = data.time.dt.dayofweek

    start_date = pd.to_datetime(date(1991, 4, 1)) #where the dataset beggins
    data.loc[:, 'days_since_beg_of_time'] = [(a_date - start_date).days for a_date in data.time]

    # get values from previous samples
    #data_shift1h = data.copy().shift(1, axis=0)

    #data = CopyFeatures(data, [1, 12, 24])

    #abs(data.date_time - start_date).days
    # data.loc[:, 'is_weekend'] = (data.date_time.dt.dayofweek > 5).astype(int) # no importance to lgbm
  
    return data, features

scaler = MinMaxScaler()
data, features = feature_generation(data)

#data[features] = scaler.fit_transform(data[features])

data

time  x  y direction  congestion  hour  day_of_week  \
row_id                                                                      
0      1991-04-01 00:00:00  0  0        EB          70     0            0   
1      1991-04-01 00:00:00  0  0        NB          49     0            0   
2      1991-04-01 00:00:00  0  0        SB          24     0            0   
3      1991-04-01 00:00:00  0  1        EB          18     0            0   
4      1991-04-01 00:00:00  0  1        NB          60     0            0   
...                    ... .. ..       ...         ...   ...          ...   
848830 1991-09-30 11:40:00  2  3        NB          54    11            0   
848831 1991-09-30 11:40:00  2  3        NE          28    11            0   
848832 1991-09-30 11:40:00  2  3        SB          68    11            0   
848833 1991-09-30 11:40:00  2  3        SW          17    11            0   
848834 1991-09-30 11:40:00  2  3        WB          24    11            0   

        days_since_beg_of_time  
row_id                          
0                            0  
1                            0  
2                            0  
3                            0  
4                            0  
...                        ...  
848830                     182  
848831                     182  
848832                     182  
848833                     182  
848834                     182  

[848835 rows x 8 columns]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, OneHotEncoder
import lightgbm as lgb

# prepare the data
y = data.pop('congestion')
X = data

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [6]:
#!pip install flaml[notebook]

#!pip install --upgrade --force-reinstall flaml

In [15]:
from flaml import AutoML

automl = AutoML()

automl.fit(X,y, task="regression", metric="mse")


#mode="max"/"min"
#automl.fit(X, y, metric="mse", task="regression", time_budget=60*5, log_file_name="automl.log", n_jobs=-1, seed= 7654321)  # , ensemble= False, estimator_list=['lgbm', 'catboost', 'xgboost']


#automl.fit(X, y, metric="mse", task="regression", time_budget=3600*8, log_file_name="automl.log", n_jobs=1, eval_method='cv', n_splits=5, estimator_list=['lgbm']) #'xgboost', 'catboost', -- don't waste time with multiple algos

[flaml.automl: 03-21 22:27:56] {2068} INFO - task = regression
[flaml.automl: 03-21 22:27:56] {2070} INFO - Data split method: uniform
[flaml.automl: 03-21 22:27:56] {2074} INFO - Evaluation method: holdout
[flaml.automl: 03-21 22:27:57] {2155} INFO - Minimizing error metric: mse
[flaml.automl: 03-21 22:27:57] {2248} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 03-21 22:27:57] {2501} INFO - iteration 0, current learner lgbm


TypeError: generate_variants() takes 1 positional argument but 2 were given

In [ ]:
#automl.best_estimator
automl.best_config
#0.1896 was best of prev run

#nu0.1863


In [ ]:
test_data = pd.read_csv('data/test.csv', index_col=0)
test_data, _ = feature_generation(test_data)
test_data = pd.DataFrame(scaler.transform(test_data), columns=features, index=test_data.index)

# holdout_pred = automl.predict(test_data)
holdout_pred = automl.predict_proba(test_data)

test_data['claim'] = holdout_pred[:,1]
submission = test_data['claim']
submission.to_csv('data/holdout_predictions.csv')
#data_to_save.reset_index()
#data_to_save.to_csv('data/holdout_predictions.csv', index=False)

In [ ]:
from flaml.data import get_output_from_log

time_history, best_valid_loss_history, valid_loss_history, config_history, train_loss_history = get_output_from_log(filename='automl.log', time_budget=3600*8)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.title('Learning Curve')
plt.xlabel('Wall Clock Time (s)')
plt.ylabel('ROC_AUC')
plt.scatter(time_history, 1 - np.array(valid_loss_history))
plt.step(time_history, 1 - np.array(best_valid_loss_history), where='post')
plt.show()

In [ ]:
print("done!")